## 2.4 Model Selection 소개

In [1]:
# 학습/테스트 데이터 분리
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import KFold

# 학습/테스트 데이터의 이해

### 학습 데이터로 잘못된 예측 케이스

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
# iris 데이터 로드
iris = load_iris()

# 학습 데이터 세팅
train_data = iris.data
train_label = iris.target

In [4]:
# 디시젼트리 분류기 인스턴스 생성
dt_clf = DecisionTreeClassifier()

In [5]:
# 학습 데이터로 학습
dt_clf.fit(train_data, train_label)

DecisionTreeClassifier()

In [6]:
# 학습 데이터로 예측 수행 : 학습 데이터로 predict(테스트)를 했기 때문에 100%
pred = dt_clf.predict(train_data)
print('예측 정확도:', accuracy_score(train_label,pred))

예측 정확도: 1.0


### 테스트 데이터로 predict해야 제대로된 예측

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier( )
iris_data = load_iris()

# train_test_split 함수 : 학습, 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.3, random_state=121)

In [8]:
print(X_train.shape)
print(X_test.shape)

(105, 4)
(45, 4)


In [9]:
# 모델 학습
dt_clf.fit(X_train, y_train)

# 테스트 데이터로 예측
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9556


### Pandas DataFrame/Series도 train_test_split( )으로 분할 가능

In [10]:
# 사이킷런 예제가 아닌 실제 데이터셋은 pandas로 분석하므로 익혀놓을 것!

import pandas as pd

iris_df = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df['target'] = iris_data.target

print(iris_df.shape)
iris_df.head()

(150, 5)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [13]:
# 피쳐 정의
ftr_df = iris_df.iloc[:, :-1]
ftr_df


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [14]:
# 타겟 정의
tgt_df = iris_df.iloc[:, -1]
tgt_df

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int32

In [15]:

# train_test_split 함수 : 학습/테스트 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(ftr_df, tgt_df, 
                                                    test_size=0.3, random_state=121)

In [16]:
# 머신러닝 모델 정의
dt_clf = DecisionTreeClassifier( )

# 학습
dt_clf.fit(X_train, y_train)

# 테스트 데이터로 예측
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9556


# 교차 검증
- 과적합(overfitting) : 모델이 학습 데이터에만 과도하게 최적화되는 형상 -> 새로운 데이터 예측시 성능이 저하됨
- 과적합 방지를 위해 (데이터 편중을 막기 위해) 더 다양한 학습과 평가를 수행하는 것이 목적
- 학습 데이터 셋을 학습과 검증으로 나눠 데이터 셋 내에서 학습과 평가를 동시해하는 것 -> 일종의 모의고사!
- 교차 검증 시 ML 데이터셋 : 학습, 검증, 테스트 데이터 셋으로 분류할 수 있음

### K 폴드 
- K 개의 데이터 폴드 셋을 만든 뒤 각 K번 만큼 데이터셋에 학슴과 검증을 반복적으로 수행하는 것
- 5 폴드의 경우 5개의 검증을 한 뒤 5개의 평가결과를 평균해서 나타낸다.

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

# 데이터 로드
iris = load_iris()
label = iris.target
features = iris.data

print(features.shape)
features

(150, 4)


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [11]:
# 모델 정의
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf

DecisionTreeClassifier(random_state=156)

In [12]:
# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성.
kfold = KFold(n_splits=5)  # n=5
cv_accuracy = []           # 최종적으로는 n번의 교차검증의 평균 정확도 계산
print('붓꽃 데이터 세트 크기:', features.shape[0])

붓꽃 데이터 세트 크기: 150


In [20]:
# for문이 도는 동안 generator가 kfold된 데이터의 학습, 검증 row 인덱스를 array로 반환  
kfold.split(features)

<generator object _BaseKFold.split at 0x000001BB6F785890>

In [13]:
n_iter = 0

# KFold객체의 split( ) 호출하면 폴드 별 학습용, 검증용 테스트의 row 인덱스를 array로 반환  
for train_index, test_index  in kfold.split(features):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    # 반복 시 마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test,pred), 4)  # 정확도 : 소수점 4자리까지 구함
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


### Stratified K 폴드

- KFOLD 교차검증의 문제점 : 불균형한 데이터에는 적용이 안된다.  
- Imblanced : 특정 레이블 값이 매우 많거나 적어서 분포가 한쪽으로 치우치는 것
- 이를 해결할 방법이 StratifiedKFold : 불균형한 분포도를 가진 레이블 데이터 집합을 균형하게 섞어주고 교차검증을 진행한다.

In [14]:
import pandas as pd

# iris 데이터 로드
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# iris 타겟값 확인
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [15]:
kfold = KFold(n_splits=3)
# kfold.split(X)는 폴드 세트를 3번 반복할 때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환. 
n_iter =0
for train_index, test_index  in kfold.split(iris_df):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]  # 학습 레이블
    label_test= iris_df['label'].iloc[test_index]    # 검증 레이블
    
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())       # 학습 레이블 분포
    print('검증 레이블 데이터 분포:\n', label_test.value_counts(), '\n')  # 검증 레이블 분포

## 교차 검증: 1
학습 레이블 데이터 분포:
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64 

## 교차 검증: 2
학습 레이블 데이터 분포:
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64 

## 교차 검증: 3
학습 레이블 데이터 분포:
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64 



-> kfold를 했더니 불균형하게 학습 레이블, 검증 레이블이 들어가 있으므로 검증이 제대로 되지 않는다  
이를 해결할 방법이 StratifiedKFold : 불균형한 분포도를 가진 레이블 데이터 집합을 균형하게 섞어주고 교차검증을 진행한다.

In [16]:
from sklearn.model_selection import StratifiedKFold

# StratifiedKFold 클래스의 인스턴스 선언 : skf
skf = StratifiedKFold(n_splits=3)
n_iter=0

# StratifiedKFold 사용시 KFold와 차이점 : 레이블 값을 넣어줘서 레이블에 맞게 균일하게 분포를 맞춰준다.
for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts(), '\n')

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
1    17
2    16
Name: label, dtype: int64 

## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
2    17
1    16
Name: label, dtype: int64 

## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
2    17
0    16
Name: label, dtype: int64 



총 150개 데이터를 3폴드로 나누었으므로 학습 100 / 검증 50으로 분리된다.  
학습 100개의 데이터와 검증 50개의 데이터 모두 원래 데이터의 분포인 1/3 씩 포함할 수 있도록 StratifiedKFold가 적용되었다.  
-> StratifiedKFold 했더니 균일하게 학습 레이블, 검증 레이블이 들어가 있으므로 검증이 제대로 된다!

### 최종적으로 StratifiedKFold를 활용한 교차 검증 정확도 확인

In [17]:
from sklearn.model_selection import StratifiedKFold

dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  
for train_index, test_index  in skfold.split(features, label):
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)

    # 반복 시 마다 정확도 측정 
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산 
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 :0.94, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.98 0.94 0.98]
## 평균 검증 정확도: 0.9666666666666667


-> 아까보다 좋은 검증 정확도가 나왔다

## cross_val_score( ) : 교차검증을 보다 간편하게

폴드 세트 추출, 학습 및 예측, 평가 과정들을 한번에 수행
cross_val_score(estimator,X,y=None, scoring=None, cv=None, n_jobs=1, verbose=0, fit_params=None, pre_dispatch='2*n_jobs')
- estimator : classifier , regression 모델 (classdifier : Stratified KFold 적용)
- X: feature dataset
- y: label dataset
- scoring: 평가지표
- cv: number of cross validation
- n_jobs: Number of jobs to run in parallel (병렬처리 갯수, -1:using all processors)
- verbose: 수행 중 정보출력 (0:출력없음, 1:자세히, 2:함축적인 정보)
- pre_dispatch: Controls the number of jobs that get dispatched during parallel execution.

In [18]:
from sklearn.tree import DecisionTreeClassifier
# cross_val_score
from sklearn.model_selection import cross_val_score , cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증 세트는 3개 
scores = cross_val_score(dt_clf , data , label , scoring='accuracy', cv=3)
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


### GridSearchCV : 교차 검증 + 하이퍼 파라미터 튜닝  
GridSearchCV(estimator, param_grid, scoring=None,  refit=True, cv=None)
- estimator : classifier, regressor, pipeline
- param_grid : 그리드 서치로 튜닝할 하이퍼 파라미터를 딕셔너리 형태로 넣음
- scoring : 평가지표
- cv : 교차 검증을 위한 분할 데이터 셋의 수
- refit : True 생성 시 최적의 하이퍼 파라미터 튜닝 후 입력된 estimator 객체를 해당 파라미터로 재학습

- 하이퍼 파라미터 : 모델의 성능을 최대로 끌어올리는 학습 조건
- 하이퍼 파라미터 튜닝의 중요성 : 학습 조건을 잘 설정해야 최대의 성능을 내는 머신러닝 모델을 얻을 수 있다.

In [55]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# iris 데이터를 로드
iris = load_iris()

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.2, random_state=121)

# 모델 정의
dtree = DecisionTreeClassifier()

### hyper-parameter 들을 딕셔너리 형태로 설정
parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2,3]}

In [ ]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold 로 나누어서 테스트 수행 설정.  
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)
### refit=True 가 default : 가장 좋은 파라미터 설정으로 재 학습 시킴.  

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_dtree.fit(X_train, y_train)

In [57]:
grid_dtree.cv_results_

{'mean_fit_time': array([0.00056871, 0.00049432, 0.00065072, 0.00065748, 0.00079942,
        0.00065295]),
 'std_fit_time': array([1.26325154e-04, 4.01947295e-06, 1.44944553e-04, 4.08382253e-05,
        1.58665848e-04, 9.04048405e-05]),
 'mean_score_time': array([0.00030931, 0.00029031, 0.00042295, 0.00033903, 0.00038147,
        0.00041676]),
 'std_score_time': array([5.49178737e-05, 1.54025701e-05, 1.09174017e-04, 2.73638340e-05,
        8.19441107e-05, 9.32557035e-05]),
 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'min_samples_split': 2},
  {'max_depth': 1, 'min_samples_split': 3},
  {'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_sa

In [56]:
# GridSearchCV 결과는 cv_results_ 라는 딕셔너리로 저장됨
# 이를 DataFrame으로 변환해서 확인
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


-> 가장 좋은 hyper-parameter는 {'max_depth': 3, 'min_samples_split': 3}	

In [25]:
# (참고) GridSearchCV 결과 전체 확인
grid_dtree.cv_results_

{'mean_fit_time': array([0.00062299, 0.00051562, 0.00049472, 0.00049384, 0.00055027,
        0.00056203]),
 'std_fit_time': array([1.91088984e-04, 3.85178642e-05, 4.78423975e-06, 5.67327127e-06,
        5.41397446e-05, 4.54078739e-05]),
 'mean_score_time': array([0.00034698, 0.00027935, 0.00027466, 0.00027641, 0.00029429,
        0.00029667]),
 'std_score_time': array([1.06681118e-04, 5.47267447e-06, 1.01152436e-06, 7.37000982e-07,
        2.53145697e-05, 2.36794450e-05]),
 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'min_samples_split': 2},
  {'max_depth': 1, 'min_samples_split': 3},
  {'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_sa

In [58]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

# refit=True로 설정된 GridSearchCV 객체가 fit()을 수행 시 학습이 완료된 Estimator를 내포하고 있으므로 predict()를 통해 예측도 가능. 
pred = grid_dtree.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9750
테스트 데이터 세트 정확도: 0.9667


In [59]:
# 테스트 데이터 예측 정확도 확인
accuracy_score(y_test, pred)

0.9666666666666667

estimator 종류
1. 분류 : DecisionTreeClassifier, RandomForestClassifier, ...
2. 회귀 : LinearRegression, ...

In [60]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
# 위에서 dtree = DecisionTreeClassifier() 로 estimator를 선언했고, 이를 GridSearchCV에 넣었으므로,
estimator = grid_dtree.best_estimator_
estimator

DecisionTreeClassifier(max_depth=3)

In [61]:
# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도: 0.9667


결론:
GridSearchCV를 사용했더니 교차검증을 통해 최적의 모델 성능을 내는 하이퍼 파라미터 튜닝을 했다.
즉, 정확도가 높은 모델을 얻었다!